In [1]:
import theano
import theano.tensor as T
from theano.tensor.signal import pool

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class cnn_layer():
    def __init__(self, output_activation_func=T.nnet.relu, K_init, b_init, pool_size=(2,2)):
        '''
        K_init: Initial values for the kernel for this layer
        b_init: Initial values for the bias for this layer
        activation: RelU
        pool_size: size over which max_pool is applied
        '''
        
        self.K = theano.shared(value=K_init.astype(theano.config.floatX),
                               borrow=True
                              )
        self.b = theano.shared(value=b_init.astype(theano.config.floatX),
                               borrow=True
                              )
        
        
        self.params = [self.K, self.b]
    
    def output(self, I):
        assert I.shape[1] == K_init.shape[1]
        self.I = I
        
        conv_out = T.nnet.conv2d(I, self.K)
        pool_out = pool.pool_2d(conv_out, pool_size, ignore_border=True)
        
        #dimshuffle shuffles and makes slice broadcastable wherever 'x' is
        return(output_activation_func(pool_out + self.b.dimshuffle('x', 0, 'x', 'x')))

In [ ]:
class layer:
    def __init__(self, activation_func=T.nnet.relu, W_init, b_init):
        '''
        output = activation(WX+b)
        X is the data matrix (each column is one example)
        W takes 'n_in' inputs from the previous layer (the number of previous layers nodes)
        and has 'n_out' outputs (the number of the next layers nodes)
        '''
        n_out, n_in = W_init.shape
        self.W = theano.shared(value=W_init.astype(theano.config.floatX), borrow=True)
        #borrow=True as we allow Theano to use memory for this object (make faster)
        self.b = theano.shared(value=b_init.reshape(n_out, 1).astype(theano.config.floatX), 
                               borrow=True, 
                               broadcastable=(False, True))
        self.activation_func = activation_func
        self.params = [self.W, self.b]
        
    def output(self, X):
        '''
        Gives the output: activation(WX+b)
        '''
        pre_activation = T.dot(self.W, X) + self.b
        return(self.activation_func(pre_activation))

In [3]:
#Running CNN over MNIST

I = T.tensor4('I')
#I.shape = (batch_size, 1, 28, 28)
y = T.ivector('y')

#50 feature maps
cnn_layer_1 = cnn_layer(K_init=np.random.rand(50, 1, 5, 5),
                        b_init=np.zeros((50,), dtype=theano.config.floatX)
                       )

cnn_layer_2 = cnn_layer(K_init=np.random.rand(50, 50, 5, 5),
                        b_init=np.zeros((50,), dtype=theano.config.floatX)
                       )

#Hidden layer shape: n_out=500, n_in=number of kernel
hidden_layer_1 = layer(W_init=np.random.rand(500, 50),
                       b_init=np.zeros(500)
                      )

output_layer = layer(T.nnet.softmax, 
                     W_init=np.random.rand(10, 50), 
                     b_init=np.zeros(10)
                    )

I = cnn_layer_1.output(I)
I = cnn_layer_2.output(I).flatten(2)
I = hidden_layer_1.output(I)
I = output_layer.output(I)

